This a UFC webscrapping and dataset manipulation code for all UFC fight data from 1993 until 2020.

In [1]:
# looking at the advanced stats for a single fight 
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

fights = requests.get('http://ufcstats.com/event-details/0b5b6876c2a4723f').text
soupo = BeautifulSoup(fights, 'lxml')
data = []
for p in soupo.findAll('p', {"class":"b-fight-details__table-text"}):
    mystr=p.string
    mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
    if mystr != 'None':
        data.append(mystr)
    print('{}'.format(mystr), end = ' ')

None None None 48 40 0 0 0 0 0 0 None U-DEC 
 5 5:00 None None None 165 186 1 0 0 0 0 0 None S-DEC 
 5 5:00 None None None 12 15 1 0 2 0 1 0 None KO/TKO Punch 2 1:00 None None None 74 16 4 2 0 0 2 0 None U-DEC 
 3 5:00 None None None 40 23 1 2 0 0 2 2 None S-DEC 
 3 5:00 None None None 18 0 0 0 0 0 0 0 None KO/TKO Punches 1 2:02 None None None 15 41 8 0 0 0 3 0 None U-DEC 
 3 5:00 None None None 5 5 2 0 1 0 2 0 None SUB Arm Triangle 1 2:58 None None None 86 53 1 1 1 0 1 0 None SUB Rear Naked Choke 3 2:13 None None None 38 54 0 2 0 0 0 1 None S-DEC 
 3 5:00 None None None 17 11 0 0 0 0 0 0 None KO/TKO Punch 1 3:01 

In [3]:
# main loop for creating the advanced stats csv file

def get_winner(input_link):
	linko = []
	fight_winner = requests.get(input_link).text
	soup_winner = BeautifulSoup(fight_winner,'lxml')

	for everylink in soup_winner.findAll('div',{"class":"b-fight-details__person"}):
		linky = re.sub(r'[\ \n]{2,}', '', str(everylink.text))

		if linky[0:1] == 'W':
			if '"' in linky:
				num = linky.index('"')
				linko.append(linky[1:num])
			else:
				linko.append(linky[1:])

	#See if its a title fight

	if 'Title Bout' in soup_winner.find('i',{"class":"b-fight-details__fight-title"}).text:
		linko.append('Title Bout')

	else:
		linko.append('Non-Title Bout')

	for img in soup_winner.findAll('img'):		

		if 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/fight.png' == img.get('src'):
			linko.append("FIGHT")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/perf.png' == img.get('src'):
			linko.append("PERF")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ko.png' == img.get('src'):
			linko.append("KO")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/sub.png' == img.get('src'):
			linko.append("SUB")
	if len(linko) == 4:
		linko[2] = linko[2] + " " + linko[3]
		linko.pop()
	
	elif 'Title' in linko[0]:
		linko.insert(0,'Draw')

	elif len(linko) == 2:
		linko.append(" ")
	return(linko)



def getnames(input_link):
	z = 0
	names =[]
	fight_detail_request = requests.get(input_link).text
	soup_details = BeautifulSoup(fight_detail_request,'lxml')
	for link in soup_details.findAll('p', {"class":"b-fight-details__table-text"}):
		if z < 2:
			#link = re.sub(r'[\ \n]{2,}', '', str(link.text)
			name = link.text[1:len(link.text) -2]
			names.append(name)
			z +=1
		else:
			break

	for link in soup_details.title.findAll('h2',{"class":"b-content__title"}):
		names.append(link.text)

	wclass = soup_details.find('div',{"class":"b-fight-details__fight-head"}).text
	wclass = re.sub(r'[\ \n]{2,}', '', str(wclass))
	if 'Title' in wclass:
		weight = wclass[4:wclass.index("Title") -1]
		names.append(weight)
	else:
		names.append(wclass)
	return(names)	

#print(getnames("http://www.ufcstats.com/fight-details/fae99a4089f8abb0"))
def getevents(input_link):
	fight_detail_request = requests.get(input_link).text
	soup_details = BeautifulSoup(fight_detail_request,'lxml')
	mylist = []
	myevent = soup_details.find('h2',{"class":"b-content__title"}).text
	myevent = re.sub(r'[\ \n]{2,}', '', str(myevent))
	mylist.append(myevent)
	for link in soup_details.findAll('li',{"class":"b-list__box-list-item"}):
		mystr = re.sub(r'[\ \n]{2,}', '', str(link.text))
		if 'Location' in mystr:
			mylist.append(mystr[9:])
		elif 'Date' in mystr:
			mylist.append(mystr[5:])
		else:
			mylist.append(mystr[11:])
	return(mylist)

print(getevents("http://www.ufcstats.com/event-details/4956f60b7fa57c1a"))

def getstats(input_link):
	fight_stats = []

	fight_stats.extend(getref(input_link))
	fight_stats.extend(get_winner(input_link))
	fight_link = requests.get(input_link).text
	soup_fight = BeautifulSoup(fight_link, 'lxml')
	for stats in soup_fight.findAll('p', {"class":"b-fight-details__table-text"}):
			mystr=stats.string
			mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
			fight_stats.append(mystr)

	if fight_stats[1] == '1':
		fight_stats += 70 *['N/A']
		for i in range(51,133):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '2':
		fight_stats += 54 *['N/A']
		for i in range(71,131):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '3':
		fight_stats += 36 *['N/A']
		for i in range(91,131):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '4':
		fight_stats += 16 *['N/A']
		for i in range(110,131):
			fight_stats.insert(i,"N/A")


	return(fight_stats)



#132-53 for round 1 finises


def getref(input_link):
	#txt is the long string that is returned from soup
	txt = []
	#ref_clean is the clean appened list of things we want to use
	ref_clean = []
	fight_link = requests.get(input_link).text
	soup_ref = BeautifulSoup(fight_link, 'lxml')
	for refs in soup_ref.findAll('p', {"class":"b-fight-details__text"}):
		mystr = refs.text
		mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
		txt.append(mystr) 

	#Parce through the online long string that is fighter details

	Method_num = txt[0].index('Round')
	ref_clean.append(txt[0][7:Method_num])

	Round_num = txt[0].index('Time')
	ref_clean.append(txt[0][Method_num+6:Round_num])

	Rndnum = txt[0].index('Time format')
	ref_clean.append(txt[0][Round_num+5:Rndnum])

	Time_num = txt[0].index('Referee')
	ref_clean.append(txt[0][Rndnum+12:Time_num])

	referee = txt[0][Time_num+8:]
	ref_clean.append(referee)

	ref_clean.append(txt[1][8:])
	return(ref_clean)



#MAIN LOOP
def main():
	#main soup for events page
	source = requests.get('http://ufcstats.com/statistics/events/completed?page=all').text
	soup = BeautifulSoup(source, 'lxml')
	x=0

	with open(r'C:\Users\brock\OneDrive\Documents\quick_join.csv','w', newline='') as file:
		writer = csv.writer(file)
		csvheader = ['FIGHTER_RED','FIGHER_BLUE','WEIGHT','EVENT','DATE','STATE_COUNTRY','ATTENDENCE','FINISH','ROUND_FINISHED','TIME_FINISHED','SCHEDUELED ROUNDS','REFEREE','METHOD_OF_FINISH',
				     'WINNER','TITLE_OR_NOT','BONUS','?','?','KD_RED','KD_BLUE','SIG_STR_RED','SIG_STR_BLUE','SIG_STR_PERCENT_RED','SIG_STR_PERCENT_BLUE',
				     'TOTAL_STR_RED','TOTAL_STR_BLUE','TD_RED','TD_BLUE','TD_PERCENT_RED','TD_PERCENT_BLUE','SUB_ATT_RED','SUB_ATT_BLUE','PASSES_RED',
				     'PASSES_BLUE','REV_RED','REV_BLUE','?','?','RND1_KD_RED','RND1_KD_BLUE','RND1_SIG_STR_RED','RND1_SIG_STR_BLUE','RND1_SIG_STR_PERC_RED',
				     'RND1_SIG_STR_PERC_BLUE','RND1_TOTAL_STR_RED','RND1_TOTAL_STR_BLUE','RND1_TD_RED','RND1_TD_BLUE','RND1_TD_PERC_RED','RND1_TD_PERC_BLUE',
				     'RND1_SUB_ATT_RED','RND1_SUB_ATT_BLUE','RND1_PASSES_RED','RND1_PASSES_BLUE','RND1_REV_RED','RND1_REV_BLUE','?','?','RND2_KD_RED','RND2_KD_BLUE',
				     'RND2_SIG_STR_RED','RND2_SIG_STR_BLUE','RND2_SIG_STR_PERC_RED',
				     'RND2_SIG_STR_PERC_BLUE','RND2_TOTAL_STR_RED','RND2_TOTAL_STR_BLUE','RND2_TD_RED','RND2_TD_BLUE','RND2_TD_PERC_RED','RND2_TD_PERC_BLUE',
				     'RND2_SUB_ATT_RED','RND2_SUB_ATT_BLUE','RND2_PASSES_RED','RND2_PASSES_BLUE','RND2_REV_RED','RND2_REV_BLUE','?','?','RND3_KD_RED','RND3_KD_BLUE',
				     'RND3_SIG_STR_RED','RND3_SIG_STR_BLUE','RND3_SIG_STR_PERC_RED',
				     'RND3_SIG_STR_PERC_BLUE','RND3_TOTAL_STR_RED','RND3_TOTAL_STR_BLUE','RND3_TD_RED','RND3_TD_BLUE','RND3_TD_PERC_RED','RND3_TD_PERC_BLUE',
				     'RND3_SUB_ATT_RED','RND3_SUB_ATT_BLUE','RND3_PASSES_RED','RND3_PASSES_BLUE','RND3_REV_RED','RND3_REV_BLUE','?','?','RND4_KD_RED','RND4_KD_BLUE',
				     'RND4_SIG_STR_RED','RND4_SIG_STR_BLUE','RND4_SIG_STR_PERC_RED',
				     'RND4_SIG_STR_PERC_BLUE','RND4_TOTAL_STR_RED','RND4_TOTAL_STR_BLUE','RND4_TD_RED','RND4_TD_BLUE','RND4_TD_PERC_RED','RND4_TD_PERC_BLUE',
				     'RND4_SUB_ATT_RED','RND4_SUB_ATT_BLUE','RND4_PASSES_RED','RND4_PASSES_BLUE','RND4_REV_RED','RND4_REV_BLUE','?','?','RND5_KD_RED','RND5_KD_BLUE',
				     'RND5_SIG_STR_RED','RND5_SIG_STR_BLUE','RND5_SIG_STR_PERC_RED',
				     'RND5_SIG_STR_PERC_BLUE','RND5_TOTAL_STR_RED','RND5_TOTAL_STR_BLUE','RND5_TD_RED','RND5_TD_BLUE','RND5_TD_PERC_RED','RND5_TD_PERC_BLUE',
				     'RND5_SUB_ATT_RED','RND5_SUB_ATT_BLUE','RND5_PASSES_RED','RND5_PASSES_BLUE','RND5_REV_RED','RND5_REV_BLUE','?','?','SIG_STR_TOTAL_RED','SIG_STR_TOTAL_BLUE',
				     'SIG_STR_PERC_RED','SIG_STR_PERC_BLUE','HEAD_STR_RED','HEAD_STR_BLUE','BODY_STR_RED','BODY_STR_BLUE','LEG_STR_RED','LEG_STR_BLUE','DISTANCE_RED','DISTANCE_BLUE',
				     'CLINCH_RED','CLICH_BLUE','GROUD_RED','GROUND_BLUE','?','?','RND1_SIG_STR_RED','RND1_SIG_STR_BLUE','RND1_SIG_STR_PERC_RED','RND1_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND1_HEAD_STR_BLUE','RND1_BODY_STR_RED','RND1_BODY_STR_BLUE','RND1_LEG_STR_RED','RND1_LEG_STR_BLUE','RND1_DISTANCE_RED','RND1_DISTANCE_BLUE',
				     'RND1_CLINCH_RED','RND1_CLINCH_BLUE','RND1_GROUND_RED','RND1_GOUND_BLUE','?','?','RND2_SIG_STR_RED','RND2_SIG_STR_BLUE','RND2_SIG_STR_PERC_RED','RND2_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND2_HEAD_STR_BLUE','RND2_BODY_STR_RED','RND2_BODY_STR_BLUE','RND2_LEG_STR_RED','RND2_LEG_STR_BLUE','RND2_DISTANCE_RED','RND2_DISTANCE_BLUE',
				     'RND2_CLINCH_RED','RND2_CLINCH_BLUE','RND2_GROUND_RED','RND2_GOUND_BLUE','?','?','RND3_SIG_STR_RED','RND3_SIG_STR_BLUE','RND3_SIG_STR_PERC_RED','RND3_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND3_HEAD_STR_BLUE','RND3_BODY_STR_RED','RND3_BODY_STR_BLUE','RND3_LEG_STR_RED','RND3_LEG_STR_BLUE','RND3_DISTANCE_RED','RND3_DISTANCE_BLUE',
				     'RND3_CLINCH_RED','RND3_CLINCH_BLUE','RND3_GROUND_RED','RND3_GOUND_BLUE','?','?','RND4_SIG_STR_RED','RND4_SIG_STR_BLUE','RND4_SIG_STR_PERC_RED','RND4_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND4_HEAD_STR_BLUE','RND4_BODY_STR_RED','RND4_BODY_STR_BLUE','RND4_LEG_STR_RED','RND4_LEG_STR_BLUE','RND4_DISTANCE_RED','RND4_DISTANCE_BLUE',
				     'RND4_CLINCH_RED','RND4_CLINCH_BLUE','RND4_GROUND_RED','RND4_GOUND_BLUE','?','?','RND5_SIG_STR_RED','RND5_SIG_STR_BLUE','RND5_SIG_STR_PERC_RED','RND5_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND5_HEAD_STR_BLUE','RND5_BODY_STR_RED','RND5_BODY_STR_BLUE','RND5_LEG_STR_RED','RND5_LEG_STR_BLUE','RND5_DISTANCE_RED','RND5_DISTANCE_BLUE',
				     'RND5_CLINCH_RED','RND5_CLINCH_BLUE','RND5_GROUND_RED','RND5_GOUND_BLUE']


		writer.writerow(csvheader)
		print('writing csv')


	for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
		print(x)
        #set x to a value to the number of desired events, new events can be concatenated later as events occur

		
		x = x + 1
		if x < 1000:
				#check if its an event details link
			if "event-details" in link.get('href'):
				print(link.get('href'))
				#soup2 is the event details link
				#souping up the event link
				source2 = requests.get(link.get('href')).text
				soup2 = BeautifulSoup(source2, 'lxml')

				csvdata = getevents(link.get('href'))
				#looping through event link to get every fight-details link
				for links in soup2.findAll('a', attrs={'href': re.compile("^http://")}):

					if "fight-details" in links.get('href'):
						source3 = requests.get(links.get('href')).text
						soup3 = BeautifulSoup(source3, 'lxml')
						csvdata = getnames(links.get('href'))
						csvdata.extend(getevents(link.get('href')))
						csvdata.extend(getstats(links.get('href')))

						print(getevents(link.get('href')))

						with open(r'C:\Users\brock\OneDrive\Documents\quick_join.csv','a', newline='') as file:
							writer = csv.writer(file)
							#csv row is the stats list
							writer.writerow(csvdata)
							print('writing csv')
													
							
		else:
			break


main()

['UFC 205: Alvarez vs McGregor', 'November 12, 2016', 'New York City, New York, USA', '20,427']
writing csv
0
1
2
http://ufcstats.com/event-details/53278852bcd91e11
3
http://ufcstats.com/event-details/0b5b6876c2a4723f
['UFC 248: Adesanya vs. Romero', 'March 07, 2020', 'Las Vegas, Nevada, USA', '15,077']
writing csv
['UFC 248: Adesanya vs. Romero', 'March 07, 2020', 'Las Vegas, Nevada, USA', '15,077']
writing csv
['UFC 248: Adesanya vs. Romero', 'March 07, 2020', 'Las Vegas, Nevada, USA', '15,077']
writing csv
['UFC 248: Adesanya vs. Romero', 'March 07, 2020', 'Las Vegas, Nevada, USA', '15,077']
writing csv
['UFC 248: Adesanya vs. Romero', 'March 07, 2020', 'Las Vegas, Nevada, USA', '15,077']
writing csv
['UFC 248: Adesanya vs. Romero', 'March 07, 2020', 'Las Vegas, Nevada, USA', '15,077']
writing csv
['UFC 248: Adesanya vs. Romero', 'March 07, 2020', 'Las Vegas, Nevada, USA', '15,077']
writing csv
['UFC 248: Adesanya vs. Romero', 'March 07, 2020', 'Las Vegas, Nevada, USA', '15,077']
wr

['UFC Fight Night: Blaydes vs. Dos Santos', 'January 25, 2020', 'Raleigh, North Carolina, USA', '14,533']
writing csv
9
http://ufcstats.com/event-details/0c1773639c795466
['UFC 246: McGregor vs. Cowboy', 'January 18, 2020', 'Las Vegas, Nevada, USA', '19,040']
writing csv
['UFC 246: McGregor vs. Cowboy', 'January 18, 2020', 'Las Vegas, Nevada, USA', '19,040']
writing csv
['UFC 246: McGregor vs. Cowboy', 'January 18, 2020', 'Las Vegas, Nevada, USA', '19,040']
writing csv
['UFC 246: McGregor vs. Cowboy', 'January 18, 2020', 'Las Vegas, Nevada, USA', '19,040']
writing csv
['UFC 246: McGregor vs. Cowboy', 'January 18, 2020', 'Las Vegas, Nevada, USA', '19,040']
writing csv
['UFC 246: McGregor vs. Cowboy', 'January 18, 2020', 'Las Vegas, Nevada, USA', '19,040']
writing csv
['UFC 246: McGregor vs. Cowboy', 'January 18, 2020', 'Las Vegas, Nevada, USA', '19,040']
writing csv
['UFC 246: McGregor vs. Cowboy', 'January 18, 2020', 'Las Vegas, Nevada, USA', '19,040']
writing csv
['UFC 246: McGregor v

['UFC Fight Night: Zabit vs. Kattar', 'November 09, 2019', 'Moscow, Moscow, Russia', '11,305']
writing csv
['UFC Fight Night: Zabit vs. Kattar', 'November 09, 2019', 'Moscow, Moscow, Russia', '11,305']
writing csv
['UFC Fight Night: Zabit vs. Kattar', 'November 09, 2019', 'Moscow, Moscow, Russia', '11,305']
writing csv
['UFC Fight Night: Zabit vs. Kattar', 'November 09, 2019', 'Moscow, Moscow, Russia', '11,305']
writing csv
['UFC Fight Night: Zabit vs. Kattar', 'November 09, 2019', 'Moscow, Moscow, Russia', '11,305']
writing csv
['UFC Fight Night: Zabit vs. Kattar', 'November 09, 2019', 'Moscow, Moscow, Russia', '11,305']
writing csv
15
http://ufcstats.com/event-details/fd87b1bbfcde9d5e
['UFC 244: Masvidal vs. Diaz', 'November 02, 2019', 'New York City, New York, USA', '20,143']
writing csv
['UFC 244: Masvidal vs. Diaz', 'November 02, 2019', 'New York City, New York, USA', '20,143']
writing csv
['UFC 244: Masvidal vs. Diaz', 'November 02, 2019', 'New York City, New York, USA', '20,143'

['UFC Fight Night: Hermansson vs. Cannonier', 'September 28, 2019', 'Copenhagen, Denmark', '12,767']
writing csv
['UFC Fight Night: Hermansson vs. Cannonier', 'September 28, 2019', 'Copenhagen, Denmark', '12,767']
writing csv
['UFC Fight Night: Hermansson vs. Cannonier', 'September 28, 2019', 'Copenhagen, Denmark', '12,767']
writing csv
['UFC Fight Night: Hermansson vs. Cannonier', 'September 28, 2019', 'Copenhagen, Denmark', '12,767']
writing csv
['UFC Fight Night: Hermansson vs. Cannonier', 'September 28, 2019', 'Copenhagen, Denmark', '12,767']
writing csv
['UFC Fight Night: Hermansson vs. Cannonier', 'September 28, 2019', 'Copenhagen, Denmark', '12,767']
writing csv
['UFC Fight Night: Hermansson vs. Cannonier', 'September 28, 2019', 'Copenhagen, Denmark', '12,767']
writing csv
21
http://ufcstats.com/event-details/94a5aaf573f780ad
['UFC Fight Night: Rodriguez vs. Stephens', 'September 21, 2019', 'Mexico City, Distrito Federal, Mexico', '10,112']
writing csv
['UFC Fight Night: Rodrigu

['UFC 241: Cormier vs. Miocic 2', 'August 17, 2019', 'Anaheim, California, USA', '17,304']
writing csv
['UFC 241: Cormier vs. Miocic 2', 'August 17, 2019', 'Anaheim, California, USA', '17,304']
writing csv
26
http://ufcstats.com/event-details/e5d03e4d966126bd
['UFC Fight Night: Shevchenko vs. Carmouche 2', 'August 10, 2019', 'Montevideo, Uruguay', '9,225']
writing csv
['UFC Fight Night: Shevchenko vs. Carmouche 2', 'August 10, 2019', 'Montevideo, Uruguay', '9,225']
writing csv
['UFC Fight Night: Shevchenko vs. Carmouche 2', 'August 10, 2019', 'Montevideo, Uruguay', '9,225']
writing csv
['UFC Fight Night: Shevchenko vs. Carmouche 2', 'August 10, 2019', 'Montevideo, Uruguay', '9,225']
writing csv
['UFC Fight Night: Shevchenko vs. Carmouche 2', 'August 10, 2019', 'Montevideo, Uruguay', '9,225']
writing csv
['UFC Fight Night: Shevchenko vs. Carmouche 2', 'August 10, 2019', 'Montevideo, Uruguay', '9,225']
writing csv
['UFC Fight Night: Shevchenko vs. Carmouche 2', 'August 10, 2019', 'Montev

...

['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
['UFC 41: Onslaught', 'February 28, 2003', 'Atlantic City, New Jersey, USA', '13,707']
writing csv
470
http://ufcstats.com/event-details/e06fd1260ac865a7
['UFC 40: Vendetta', 'November 22, 2002', 'Las Vegas, 

['UFC 32: Showdown in the Meadowlands', 'June 29, 2001', 'East Rutherford, New Jersey, USA', '11,492']
writing csv
['UFC 32: Showdown in the Meadowlands', 'June 29, 2001', 'East Rutherford, New Jersey, USA', '11,492']
writing csv
['UFC 32: Showdown in the Meadowlands', 'June 29, 2001', 'East Rutherford, New Jersey, USA', '11,492']
writing csv
['UFC 32: Showdown in the Meadowlands', 'June 29, 2001', 'East Rutherford, New Jersey, USA', '11,492']
writing csv
['UFC 32: Showdown in the Meadowlands', 'June 29, 2001', 'East Rutherford, New Jersey, USA', '11,492']
writing csv
['UFC 32: Showdown in the Meadowlands', 'June 29, 2001', 'East Rutherford, New Jersey, USA', '11,492']
writing csv
['UFC 32: Showdown in the Meadowlands', 'June 29, 2001', 'East Rutherford, New Jersey, USA', '11,492']
writing csv
480
http://ufcstats.com/event-details/20bd6c3e03c46ee6
['UFC 31: Locked and Loaded', 'May 04, 2001', 'Atlantic City, New Jersey, USA', '5,000']
writing csv
['UFC 31: Locked and Loaded', 'May 04, 

['UFC 22: Only One Can be Champion', 'September 24, 1999', 'Lake Charles, Louisiana, USA', '4,000']
writing csv
['UFC 22: Only One Can be Champion', 'September 24, 1999', 'Lake Charles, Louisiana, USA', '4,000']
writing csv
['UFC 22: Only One Can be Champion', 'September 24, 1999', 'Lake Charles, Louisiana, USA', '4,000']
writing csv
['UFC 22: Only One Can be Champion', 'September 24, 1999', 'Lake Charles, Louisiana, USA', '4,000']
writing csv
['UFC 22: Only One Can be Champion', 'September 24, 1999', 'Lake Charles, Louisiana, USA', '4,000']
writing csv
['UFC 22: Only One Can be Champion', 'September 24, 1999', 'Lake Charles, Louisiana, USA', '4,000']
writing csv
['UFC 22: Only One Can be Champion', 'September 24, 1999', 'Lake Charles, Louisiana, USA', '4,000']
writing csv
['UFC 22: Only One Can be Champion', 'September 24, 1999', 'Lake Charles, Louisiana, USA', '4,000']
writing csv
490
http://ufcstats.com/event-details/1a1a4d7a29041d77
['UFC 21: Return of the Champions', 'July 16, 199

['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
['UFC 14: Showdown', 'July 27, 1997', 'Birmingham, Alabama, USA', '4,800']
writing csv
500
http://ufcstats.com/event-details/5bd533d50c8e7b8a
['UFC 13: The Ultimate Force', 'May 30, 1997', 'Augusta, Georgia, USA', '5,100']
writing csv
['UFC 13: The Ultimate Force', 'May 30, 1997', 'Augusta, Georgia, USA

['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
writing csv
['UFC 7: The Brawl in Buffalo', 'September 08, 1995', 'Buffalo, New York, USA', '9,000']
wr

In [ ]:
advanced = pd.read_csv(r'C:\Users\brock\OneDrive\Documents\quick_join.csv')
advanced

In [3]:
#importing required CSVs and renaming a few column errors, also setting dataset display widths

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
advanced = pd.read_csv(r'C:\Users\brock\OneDrive\Documents\quick_join.csv')
advanced.rename({'TITLE_OR_NOT': 'METHOD_OF_FINISH', 'METHOD_OF_FINISH': 'WINNER','WINNER':'TITLE_OR_NOT'}, axis=1)
advanced

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
0,Israel Adesanya,Yoel Romero,Middleweight,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Dan Miragliotta,Ron McCarthy48 - 47.Sal D'amato48 - 47.Chris L...,Israel Adesanya,Title Bout,,None,None,0.0,0

In [4]:
# All duplicated columns with the bonus as None have column formatting issue, columns are shifted to the right by 1.

p1 = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')].iloc[:,0:16]
p1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS
111,Marina Rodriguez,Cynthia Calvillo,Women's Strawweight Bout,UFC Fight Night: Overeem vs. Rozenstruik,"December 07, 2019","Washington, DC, USA","10,816",Decision - Majority,3,5:00,3 Rnd (5-5-5),Fernando Yamasaki,Sal D'amato28 - 28.David Braslow28 - 28.Chris ...,Draw,Non-Title Bout,None
115,Cody Stamann,Song Yadong,Bantamweight Bout,UFC Fight Night: Overeem vs. Rozenstruik,"December 07, 2019","Washington, DC, USA","10,816",Decision - Majority,3,5:00,3 Rnd (5-5-5),Kevin Mulhall,David Braslow28 - 28.Steve Rados28 - 28.Sal D'...,Draw,Non-Title Bout,None
125,Mauricio Rua,Paul Craig,Light Heavyweight Bout,UFC Fight Night: Blachowicz vs. Jacare,"November 16, 2019","Sao Paulo, Sao Paulo, Brazil","10,344",Decision - Split,3,5:00,3 Rnd (5-5-5),Jerin Valel,Guilherme Bravo28 - 28.Marcelo Vilhena28 - 29....,Draw,Non-Title Bout,None
175,Greg Hardy,Ben Sosoli,Heavyweight Bout,UFC Fight Night: Reyes vs. Weidman,"October 18, 2019","Boston, Massachusetts, USA","12,066",Overturned,3,5:00,3 Rnd (5-5-5),Bryan Miner,Illegal Inhaler Use by HardyMacel Varella29 - ...,Draw,Non-Title Bout,None
225,Yair Rodriguez,Jeremy Stephens,Featherweight Bout,UFC Fight Night: Rodriguez vs. Stephens,"September 21, 2019","Mexico City, Distrito Federal, Mexico","10,112",Could Not Continue,1,0:15,5 Rnd (5-5-5-5-5),Herb Dean,Eye poke via Rodriguez,Draw,Non-Title Bout,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5416,Tim Lajci,Ron Waterma,Heavyweight Bout,UFC 22: Only One Can be Champion,"September 24, 1999","Lake Charles, Louisiana, USA","4,000",Decision - Unanimous,3,5:00,3 Rnd (5-5-5),John McCarthy,NaN,Draw,Non-Title Bout,None
5423,Jens Pulve,Alfonso Alcare,Lightweight Bout,UFC 22: Only One Can be Champion,"September 24, 1999","Lake Charles, Louisiana, USA","4,000",Decision - Majority,2,5:00,2 Rnd (5-5),NaN,NaN,Draw,Non-Title Bout,None
5483,Kazushi Sakurab,Marcus Silveir,Heavyweight Bout,UFC - Ultimate Japan,"December 21, 1997","Yokohama, Kanagawa, Japan","5,000",Overturned,1,1:51,1 Rnd + OT (12-3),John McCarthy,Early Stoppage,Draw,Non-Title Bout,None
5574,Ken Shamroc,Oleg Taktaro,UFC Superfight Championship Bout,UFC 7: The Brawl in Buffalo,"September 08, 1995","Buffalo, New York, USA","9,000",Other,2,3:00,1 Rnd + OT (30-3),John McCarthy,Time Expired,Draw,Non-Title Bout,None


In [5]:
# Grabing the 2nd half of the data and shifting it over 1 column to the left to fix the formatting issue
p2 = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')].iloc[:,16:].shift(1,axis='columns',fill_value = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')]['?.1'])
p2

,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
111,0,None,0,0,56 of 140,78 of 130,40%,60%,61 of 145,112 of 173,0 of 0,3 of 4,0%,75%,0,0,0,7,0,0,None,None,0,0,22 of 53,21 of 32,41%,65%,27 of 58,25 of 37,0 of 0,2 of 2,0%,100%,0,0,0,3,0,0,None,None,0,0,29 of 71,13 of 39,40%,33%,29 of 71,14 of 40,0 of 0,0 of 1,0%,0%,0,0,0,0,0,0,None,None,0,0,5 of 16,44 of 59,31%,74%,5 of 16,73 of 96,0 of 0,1 of 1,0%,100%,0,0,0,4,0,0,None,None,0,56 of 140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [6]:
#join the 2 halfs together
df1 = pd.concat([p1,p2], sort = False, axis = 1)
df1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
111,Marina Rodriguez,Cynthia Calvillo,Women's Strawweight Bout,UFC Fight Night: Overeem vs. Rozenstruik,"December 07, 2019","Washington, DC, USA","10,816",Decision - Majority,3,5:00,3 Rnd (5-5-5),Fernando Yamasaki,Sal D'amato28 - 28.David Braslow28 - 28.Chris ...,Draw,Non-Title

In [7]:
#grabbing the duplicated values with proper column formatting
p3 = advanced[(advanced.duplicated()) & (advanced['BONUS'] != 'None')]
p3

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
1226,Lando Vannata,Bobby Green,Lightweight Bout,UFC 216: Ferguson vs. Lee,"October 07, 2017","Las Vegas, Nevada, USA","10,638",Decision - Split,3,5:00,3 Rnd (5-5-5),Herb Dean,Derek Cleary28 - 28.Jeff Mullen29 - 27.Marcos ...,Draw,Non-Title Bout,FIGHT,None,None,1.0,0,104 of 232,

In [8]:
# adding them to the complete dataset containing all duplicated values
df1 = pd.concat([df1,p3])
df1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
111,Marina Rodriguez,Cynthia Calvillo,Women's Strawweight Bout,UFC Fight Night: Overeem vs. Rozenstruik,"December 07, 2019","Washington, DC, USA","10,816",Decision - Majority,3,5:00,3 Rnd (5-5-5),Fernando Yamasaki,Sal D'amato28 - 28.David Braslow28 - 28.Chris ...,Draw,Non-Title

In [9]:
# filling NAs as 0
advanced.fillna(value = 0,inplace = True)

In [10]:
# dropping all duplicated items then adding our corrected dataset 
advanced.drop_duplicates(keep = False,inplace = True)
advanced = pd.concat([advanced , df1])

In [11]:
#Appears to be an issue with some columns being shifted from row 138 and onward (looking to fix this in the main loop in the future)
s1 = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False)].iloc[:,0:137]
s1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12
5,Sean O'Malley,Jose Quinonez,Bantamweight Bout,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",KO/TKO,1,2:02,3 Rnd (5-5-5),Dan Miragliotta,Punches to Head On Ground,Sean O'Malley,Non-Title Bout,,None,None,0,0,18 of 22,0 of 11,81%,0%,18 of 22,0 of 11,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0,0,18 of 22,0 of 11,81%,0%,18 of 22,0 of 11,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Rodolfo Vieira,Saparbeg Safarov,Middleweight Bout,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",Submission,1,2:58,3 Rnd (5-5-5),Jason Herzog,Arm Triangle From Mount,Rodolfo Vieira,Non-Title Bout,,None,None,0,0,5 of 10,5 of 10,50%,50%,10 of 15,7 of 12,2 of 2,0 of 0,100%,0%,1,0,2,0,0,0,None,None,0,0,5 of 10,5 of 10,50%,50%,10 of 15,7 of 12,2 of 2,0 of 0,100%,0%,1,0,2,0,0,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,Danaa Batgerel,Guido Cannetti,Bantamweight Bout,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",KO/TKO,1,3:01,3 Rnd (5-5-5),Chris Tognoni,Punch to Head At Distance,Danaa Batgerel,Non-Title Bout,,None,None,1,0,17 of 33,11 of 23,51%,47%,17 of 33,11 of 23,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,1,0,17 of 33,11 of 23,51%,47%,17 of 33,11 of 23,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,Felicia Spencer,Zarah Fairn,Women's Featherweight Bout,UFC Fight Night: Benavidez vs. Figueiredo,"February 29, 2020","Norfolk, Virginia, USA","7,098",KO/TKO,1,3:37,3 Rnd (5-5-5),Dan Miragliotta,Punches to Head From Mount,Felicia Spencer,Non-Title Bout,,None,None,0,0,40 of 64,13 of 28,62%,46%,68 of 98,14 of 29,1 of 2,0 of 0,50%,

In [12]:
#shifting the data so the columns are in the proper loations
s2 = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False)].iloc[:,137:].shift(-2,axis =1 )
s2

,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
5,0,18 of 22,0 of 11,81%,0%,9 of 11,0 of 7,6 of 7,0 of 1,3 of 4,0 of 3,13 of 17,0 of 11,0 of 0,0 of 0,5 of 5,0 of 0,None,None,18 of 22,0 of 11,81%,0%,9 of 11,0 of 7,6 of 7,0 of 1,3 of 4,0 of 3,13 of 17,0 of 11,0 of 0,0 of 0,5 of 5,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
7,0,5 of 10,5 of 10,50%,50%,5 of 10,2 of 7,0 of 0,1 of 1,0 of 0,2 of 2,0 of 3,5 of 10,0 of 0,0 of 0,5 of 7,0 of 0,None,None,5 of 10,5 of 10,50%,50%,5 of 10,2 of 7,0 of 0,1 of 1,0 of 0,2 of 2,0 of 3,5 of 10,0 of 0,0 of 0,5 of 7,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
10,0,17 of 33,11 of 23,51%,47%,16 of 29,0 of 7,0 of 2,3 of 6,1 of 2,8 of 10,12 of 27,9 of 21,0 of 1,2 of 2,5 of 5,0 of 0,None,None,17 of 33,11 of 23,51%,47%,16 of 29,0 of 7,0 of 2,3 of 6,1 of 2,8 of 10,12 of 27,9 of 21,0 of 1,2 of 2,5 of 5,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
12,0,40 of 64,13 of 28,62%,46%,38 of 61,12 of 27,1 of 1,1 of 1,1 of 2,0 of 0,9 of 24,12 of 26,1 of 1,1 of 2,30 of 39,0 of 0,None,None,40 of 64,13 of 28,62%,46%,38 of 61,12 of 27,1 of 1,1 of 1,1 of 2,0 of 0,9 of 24,12 of 26,1 of 1,1 of 2,30 of 39,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
13,0,5 of 11,8 of 12,45%,66%,3 of 8,6 of 10,0 of 1,1 of 1,2 of 2,1 of 1,5 of 11,8 of 12,0 of 0,0 of 0,0 of 0,0 of 0,None,None,5 of 11,8 of 12,45%,66%,3 of 8,6 of 10,0 of 1,1 of 1,2 of 2,1 of 1,5 of 11,8 of 12,0 of 0,0 of 0,0 of 0,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [13]:
#adding the 2 peices together
df2 = pd.concat([s1,s2], axis = 1)
df2

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
5,Sean O'Malley,Jose Quinonez,Bantamweight Bout,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",KO/TKO,1,2:02,3 Rnd (5-5-5),Dan Miragliotta,Punches to Head On Ground,Sean O'Malley,Non-Title Bout,,None,None,0,0,18 of 22,0 of 11,81%,0%,18 of 22,0 o

In [15]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5552 entries, 0 to 4864
Columns: 244 entries, FIGHTER_RED to RND5_GOUND_BLUE
dtypes: object(244)
memory usage: 10.4+ MB


In [16]:
# removing all incorrectly formatted columns and then adding our corrected dataset
advanced = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False) == False]
advanced = pd.concat([advanced,df2])

In [17]:
#Quick test
advanced[advanced['WINNER'] == 'Draw']

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
111,Marina Rodriguez,Cynthia Calvillo,Women's Strawweight Bout,UFC Fight Night: Overeem vs. Rozenstruik,"December 07, 2019","Washington, DC, USA","10,816",Decision - Majority,3,5:00,3 Rnd (5-5-5),Fernando Yamasaki,Sal D'amato28 - 28.David Braslow28 - 28.Chris ...,Draw,Non-Title

In [18]:
# similar column formation issue, creating corrected versions of the data
t1 = advanced[advanced['GROUND_BLUE'] == 'None'].iloc[:,0:137]
t2 = advanced[advanced['GROUND_BLUE'] == 'None'].iloc[:,137:].shift(1,axis=1)

In [19]:
df3 = pd.concat([t1,t2],axis=1)

In [20]:
advanced = advanced[advanced['GROUND_BLUE'] != 'None']

In [21]:
advanced = pd.concat([advanced,df3])
advanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5552 entries, 0 to 5414
Columns: 244 entries, FIGHTER_RED to RND5_GOUND_BLUE
dtypes: object(244)
memory usage: 10.4+ MB


In [25]:
# removing unneeded columns
for i in advanced.columns:
    if i[0] == "?" or i[0:3] == 'Unn':
        print(i)
        advanced.drop(i,axis = 1,inplace=True)

?
?.1
?.2
?.3
?.4
?.5
?.6
?.7
?.8
?.9
?.10
?.11
?.12
?.13
?.14
?.15
?.16
?.17
?.18
?.19
?.20
?.21
?.22
?.23


In [26]:
#reseting the index 
advanced.reset_index(inplace = True)

In [28]:
#creating the loser stat, if the fighter isn't a winner and the fight wasn't a draw, that fighter was a loser
advanced['LOSER'] = 0
count = 0
for i in advanced[['FIGHTER_RED','FIGHER_BLUE','WINNER']].values:
    if i[0] not in i[2]:
        advanced['LOSER'].loc[count] = i[0]
    elif i[1] not in i[2]:
        advanced['LOSER'].loc[count] = i[1]
    else:
        advanced['LOSER'].loc[count] = 'Draw'
        
    print(count)
        
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222


3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946
3947
3948


5506
5507
5508
5509
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
5535
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5546
5547
5548
5549
5550
5551


In [29]:
#fill NAs as 0
advanced.fillna(value = 0,inplace = True)

In [30]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5552 entries, 0 to 5551
Columns: 222 entries, index to LOSER
dtypes: float64(35), int64(1), object(186)
memory usage: 9.4+ MB


In [34]:
#writing to csv
advanced.to_csv(r'C:\Users\brock\OneDrive\Documents\UFC_Fight_Stats2.csv')